<a href="https://colab.research.google.com/github/CassioML/cassio-website/blob/main/docs/frameworks/langchain/.colab/colab_qa-maximal-marginal-relevance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# VectorStore/QA, MMR support

_**NOTE:** this uses Cassandra's "Vector Search" capability.
Make sure you are connecting to a vector-enabled database for this demo._

Cassandra's `VectorStore` allows for Vector Search with the **Maximal Marginal Relevance (MMR)** algorithm.

This is a search criterion that instead of just selecting the _k_ stored documents most relevant to the provided query, first identifies a larger pool of relevant results, and then singles out _k_ of them so that they carry as diverse information between them as possible.

In this way, when the stored text fragments are likely to be redundant, you can optimize token usage and help the models give more comprehensive answers.

_This is very useful, for instance, if you are building a QA chatbot on past Support chat recorded interactions._

First prepare a connection to a vector-search-capable Cassandra and initialize the required LLM and embeddings:

In [1]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.vectorstores import Cassandra

A database connection is needed. Let's insert the AstraDB credentials:

In [4]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_ID"] = input("Enter your Astra DB ID: ")
os.environ["ASTRA_DB_APPLICATION_TOKEN"] = getpass("Enter your Astra DB Token: ")
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

Now instantiate the LLM and the embeddings service.

In [ ]:
import os

from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
embeddings = OpenAIEmbeddings()

## Create the store

Create a (Cassandra-backed) `VectorStore` and the corresponding LangChain `VectorStoreIndexWrapper`

In [7]:
import os

import cassio

cassio.init(
    database_id=os.environ["ASTRA_DB_ID"],
    token=os.environ["ASTRA_DB_APPLICATION_TOKEN"],
)

cassandra_vstore = Cassandra(
    embedding=embeddings, table_name="mmr", session=None, keyspace="default_keyspace"
)
index = VectorStoreIndexWrapper(vectorstore=cassandra_vstore)

This command simply resets the store in case you want to run this demo repeatedly:

In [8]:
cassandra_vstore.clear()

## Populate the index


Notice that the first four sentences express the same concept, while the **fifth** adds a new detail:

In [9]:
BASE_SENTENCE_0 = (
    "The frogs and the toads were meeting in the night for a party under the moon."
)

BASE_SENTENCE_1 = (
    "There was a party under the moon, that all toads, "
    "with the frogs, decided to throw that night."
)

BASE_SENTENCE_2 = (
    'And the frogs and the toads said: "Let us have a party '
    'tonight, as the moon is shining".'
)

BASE_SENTENCE_3 = (
    "I remember that night... toads, along with frogs, "
    "were all busy planning a moonlit celebration."
)

DIFFERENT_SENTENCE = (
    "For the party, frogs and toads set a rule: everyone was to wear a purple hat."
)

Insert the three into the index, specifying "sources" while you're at it (it will be useful later):

In [10]:
texts = [
    BASE_SENTENCE_0,
    BASE_SENTENCE_1,
    BASE_SENTENCE_2,
    BASE_SENTENCE_3,
    DIFFERENT_SENTENCE,
]
metadatas = [
    {"source": "Barney's story at the pub"},
    {"source": "Barney's story at the pub"},
    {"source": "Barney's story at the pub"},
    {"source": "Barney's story at the pub"},
    {"source": "The chronicles at the village library"},
]


ids = cassandra_vstore.add_texts(
    texts,
    metadatas=metadatas,
)
print("\n".join(ids))

46c33fe2a3634ad79856006fc54176d5
12a2f838099642fe8bf365e228fb369c
2ed56f27a33e41fa94748769c8bc05c3
04b17c6b685a4e3c9bd2758ee7d40f9b
b825da62352b4e93867ace8d87b90db8


## Query the store

Here is the question you'll use to query the index:

In [11]:
QUESTION = "Tell me about the party that night."

### Query with "similarity" search type

If you ask for two matches, you will get the two documents most related to the question. But in this case this is something of a waste of tokens:

In [12]:
matches_sim = cassandra_vstore.search(QUESTION, search_type="similarity", k=2)
for i, doc in enumerate(matches_sim):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "There was a party under the moon, that all toads, with the frogs, decided to throw that night."
[ 1]: "I remember that night... toads, along with frogs, were all busy planning a moonlit celebration."


### Query with MMR

Now, here's what happens with the MMR search type.

_(Not shown here: you can tune the size of the results pool for the first step of the algorithm.)_

In [13]:
matches_mmr = cassandra_vstore.search(QUESTION, search_type="mmr", k=2)
for i, doc in enumerate(matches_mmr):
    print(f'[{i:2}]: "{doc.page_content}"')

[ 0]: "There was a party under the moon, that all toads, with the frogs, decided to throw that night."
[ 1]: "For the party, frogs and toads set a rule: everyone was to wear a purple hat."


## Query the index

Currently, LangChain's higher "index" abstraction does not allow to specify the search type, nor the number of matches subsequently used in creating the answer. So, by running this command you get an answer, all right.

In [14]:
# (implicitly) by similarity
print(index.query(QUESTION, llm=llm))

 The frogs and toads were having a party under the moon that night. They were busy planning and celebrating together.


You can request the question-answering process to provide references (as long as you annotated all input documents with a `source` metadata field):

In [15]:
response_sources = index.query_with_sources(QUESTION, llm=llm)
print("Automatic chain (implicitly by similarity):")
print(f'  ANSWER : {response_sources["answer"].strip()}')
print(f'  SOURCES: {response_sources["sources"].strip()}')

Automatic chain (implicitly by similarity):
  ANSWER : The frogs and toads were planning a party under the moon that night.
  SOURCES: Barney's story at the pub


Here the default is to fetch _four_ documents ... so that the only other text actually carrying additional information is left out!

### The QA Process behind the scenes

In order to exploit the MMR search in end-to-end question-answering pipelines, you need to recreate and manually tweak the steps behind the `query` or `query_with_sources` methods. This takes just a few lines.

First you need a few additional modules:

In [16]:
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain.chains.retrieval_qa.base import RetrievalQA

You are ready to run two QA chains, identical in all respects (especially in the number of results to fetch, two), except the `search_type`:

#### Similarity-based QA

In [17]:
# manual creation of the "retriever" with the 'similarity' search type
retriever = cassandra_vstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 2,
        # ...
    },
)
# Create a "RetrievalQA" chain
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
)
# Run it and print results
response = chain.run(QUESTION)
print(response)

 The party was held under the moon and was planned by both toads and frogs.


#### MMR-based QA

In [18]:
# manual creation of the "retriever" with the 'MMR' search type
retriever = cassandra_vstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2,
        # ...
    },
)
# Create a "RetrievalQA" chain
chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)
# Run it and print results
response = chain.run(QUESTION)
print(response)

 The party was held under the moon and was attended by both frogs and toads. Everyone was required to wear a purple hat.


#### Answers with sources

You can run the variant of these chains that also returns the source for the documents used in preparing the answer, which makes it even more obvious:

In [19]:
retriever = cassandra_vstore.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 2,
        # ...
    },
)
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    retriever=retriever,
)
response = chain({chain.question_key: QUESTION})
print("Similarity-based chain:")
print(f'  ANSWER : {response["answer"].strip()}')
print(f'  SOURCES: {response["sources"].strip()}')

Similarity-based chain:
  ANSWER : The toads and frogs were planning a moonlit celebration.
  SOURCES: Barney's story at the pub


In [20]:
retriever = cassandra_vstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 2,
        # ...
    },
)

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm,
    retriever=retriever,
)
response = chain({chain.question_key: QUESTION})
print("MMR-based chain:")
print(f'  ANSWER : {response["answer"].strip()}')
print(f'  SOURCES: {response["sources"].strip()}')

MMR-based chain:
  ANSWER : The party that night was thrown by frogs and toads, and everyone was required to wear a purple hat.
  SOURCES: Barney's story at the pub, The chronicles at the village library
